In [51]:
import tensorflow as tf
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
print(tf.version.VERSION)

from win32gui import FindWindow, SetForegroundWindow, GetClientRect, ClientToScreen
import pyautogui
import cv2

import tkinter as tk
from PIL import Image, ImageTk

from threading import Thread

2.1.0


In [52]:
"""
from win32gui import GetWindowText, EnumWindows
def enum_window_titles():
	#returns all open window class names
	def callback(handle, data):
		titles.append(GetWindowText(handle))
	titles = []
	EnumWindows(callback, None)
	return titles
print(enum_window_titles())
enum_window_titles()
"""

'\nfrom win32gui import GetWindowText, EnumWindows\ndef enum_window_titles():\n\t#returns all open window class names\n\tdef callback(handle, data):\n\t\ttitles.append(GetWindowText(handle))\n\ttitles = []\n\tEnumWindows(callback, None)\n\treturn titles\nprint(enum_window_titles())\nenum_window_titles()\n'

In [53]:
checkpoint_path = "C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights/saved_model.pb"
DATADIR5 = r"D:\K14\Dataset\Predicitions.pickle"
checkpoint_dir = os.path.dirname(checkpoint_path)

WindowClassName0 = 'Rainbow Six'
WindowClassName1 = 'J2 - Jupyter Notebook - Google Chrome'

PredImage = None

PredImage0 = None
ImageTesten0 = None
PredImage1 = None
ImageTesten1 = None

In [54]:
def load_K14():
    #model = create_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    #model.summary()
    model = tf.keras.models.load_model(checkpoint_dir)
    #model.summary()

    configproto = tf.compat.v1.ConfigProto() 
    configproto.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=configproto) 
    tf.compat.v1.keras.backend.set_session(sess)
    return model

In [55]:
model = load_K14(
)

In [56]:
Template = pickle.load(open(r"D:\K14\Dataset\Predicitions.pickle", "rb"))[10]
PredImage0 = pickle.load(open(r"D:\K14\Dataset\TestImages.pickle", "rb"))[1][8]
PredImage1 = pickle.load(open(r"D:\K14\Dataset\TestImages.pickle", "rb"))[0][8]

In [57]:
def screenshot(window_title=None, lenx=512, leny=512):
	if window_title:
		hwnd = FindWindow(None, window_title)
		if hwnd:
			SetForegroundWindow(hwnd)
			x, y, x1, y1 = GetClientRect(hwnd)
			x, y = ClientToScreen(hwnd, (x, y))
			x1, y1 = ClientToScreen(hwnd, (x1 - x, y1 - y))
			# x,y,x1,y1 position and size
			x += int((x1-lenx)/2)
			x1 = lenx
			y += int((y1-leny)/2)
			y1 = leny
			im = pyautogui.screenshot(region=(x, y, x1, y1))
			gray = np.array(im)
			image = np.array(gray).reshape(-1, lenx, leny, 3)
			return image
		else:
			print('Window not found!')
	else:
		im = pyautogui.screenshot()
		gray = cv2.cvtColor(np.array(im))
		image = np.array(gray).reshape(-1, lenx, leny, 3)
		return image

In [58]:
def xyz(image):
    a = time.time()
    im_preds = model.predict_on_batch(image)
    print(time.time()-a)
    image = image.reshape(512, 512, 3)
    im_preds = tf.reshape(im_preds, [512, 512, 1]).numpy()
    #ImCompareGray(image, im_preds)
    return im_preds

In [59]:
def Coords(Pred, small):
	#cv2, Predicitons.pickle[2]=small
	Pred = (255 - cv2.inRange(np.array(Pred * 255, dtype = np.uint8), 0, 160)).astype(np.float32)
	small = np.array(small * 255, dtype = np.uint8).astype(np.float32)
	result = cv2.matchTemplate(Pred, small, cv2.TM_SQDIFF_NORMED)
	mn,_,mnLoc,_ = cv2.minMaxLoc(result)
	MPx,MPy = mnLoc
	#x and y may flipped
	return (MPx+2-256, MPy+2-256)
	#return (MPx, MPy)

In [60]:
class myThread(Thread):
	def __init__(self):
		Thread.__init__(self)
		None
	def run(self):
		# saves the images in the global list
		root = tk.Tk()
		canvas0 = tk.Canvas(root, width = 512, height = 512)
		canvas0.pack()
		canvas1 = tk.Canvas(root, width = 512, height = 512)
		canvas1.pack()
		def recursive():
			global ImageTesten0, ImageTesten1, PredImage0, PredImage1
			ImageTesten0 = ImageTk.PhotoImage(Image.fromarray((cv2.cvtColor(PredImage0,cv2.COLOR_GRAY2RGB) * 255).astype(np.uint8)))
			ImageTesten1 = ImageTk.PhotoImage(Image.fromarray(PredImage1))
			#ImageTesten = tk.PhotoImage(file=DATADIR2+random.choice(os.listdir(DATADIR2)))
			canvas0.create_image(0,0, anchor=tk.NW, image=ImageTesten0)
			canvas1.create_image(0,0, anchor=tk.NW, image=ImageTesten1)
			root.after(100, recursive)
		recursive()
		root.mainloop()

In [61]:
inst = myThread()
inst.start()

Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\8holz\anaconda3\envs\tensorflow-gpu\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-60-5b034484c6cc>", line 20, in run
    recursive()
  File "<ipython-input-60-5b034484c6cc>", line 14, in recursive
    ImageTesten0 = ImageTk.PhotoImage(Image.fromarray((cv2.cvtColor(PredImage0,cv2.COLOR_GRAY2RGB) * 255).astype(np.uint8)))
  File "C:\Users\8holz\anaconda3\envs\tensorflow-gpu\lib\site-packages\PIL\ImageTk.py", line 112, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
  File "C:\Users\8holz\anaconda3\envs\tensorflow-gpu\lib\tkinter\__init__.py", line 3545, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "C:\Users\8holz\anaconda3\envs\tensorflow-gpu\lib\tkinter\__init__.py", line 3501, in __init__
    self.tk.call(('image', 'create', imgtype, name,) + options)
RuntimeError: main thread is not in main loop

Exception ignored in: <bou

In [30]:
time.sleep(5)

In [31]:
while True:
    PredImage0 = screenshot(WindowClassName0)
    PredImage1 = PredImage0
    time.sleep(0.005)

0.0030219554901123047
0.00399017333984375
0.02293872833251953
0.0019948482513427734
0.003989458084106445
0.005983591079711914
0.002992868423461914
0.014960527420043945
0.0019948482513427734
0.006980419158935547
0.0059833526611328125
0.003988981246948242
0.005983829498291016
0.005983829498291016
0.006981611251831055
0.005983114242553711
0.005986928939819336
0.005983114242553711
0.006982088088989258
0.003990888595581055
0.00598454475402832
0.005983591079711914
0.00399017333984375
0.00299072265625
0.0029938220977783203
0.007981538772583008
0.003988742828369141
0.0049860477447509766
0.003988981246948242
0.0019948482513427734
0.006981611251831055
0.003989696502685547
0.003991842269897461
0.004987239837646484
0.004986286163330078
0.005983829498291016
0.003990650177001953
0.007983922958374023
0.004988193511962891
0.004985332489013672
0.005983591079711914
0.003988504409790039
0.00498509407043457
0.004986286163330078
0.005983829498291016
0.004986286163330078
0.003989458084106445
0.0039925575256

error: (0, 'SetForegroundWindow', 'No error message is available')

In [32]:
def Coords(Pred, small):
	#cv2, Predicitons.pickle[2]=small
	Pred = (255 - cv2.inRange(np.array(Pred * 255, dtype = np.uint8), 0, 160)).astype(np.float32)
	small = np.array(small * 255, dtype = np.uint8).astype(np.float32)
	result = cv2.matchTemplate(Pred, small, cv2.TM_SQDIFF_NORMED)
	mn,_,mnLoc,_ = cv2.minMaxLoc(result)
	MPx,MPy = mnLoc
	#x and y may flipped
	return (MPx+2-256, MPy+2-256)
	#return (MPx, MPy)

In [33]:
Template = pickle.load(open(r"D:\K14\Dataset\Predicitions.pickle", "rb"))[10]

In [34]:
c = Coords(PredImage, Template)

In [35]:
image = cv2.rectangle(PredImage, (c[0]+256, c[1]+256), (c[0]+256+2, c[1]+256+2), (0,255,0), 2)

In [ ]:
print(1)